# Prepare Notebook

To run this code, written at the very end of Chapter 3, you need a working empty database.

To move the project into a valid state, please use the command `git chapter [chapter-number]` to find a valid commit. A commit at the end of Chapter 3, or any commit in Chapters 4-9, should work just fine. Use `git checkout [commit]` to change the state of the project. Once there, delete the database (if it exists).

    $ rm db.sqlite3

To (re-)create the database:

    $ ./manage.py migrate
    
Please see the Read Me file or the actual book for more details.

In [31]:
from datetime import date
from organizer.models import Tag, Startup, NewsLink
from blog.models import Post

# Interacting With the Database

In [3]:
edut = Tag(name='Education', slug='education')

In [4]:
edut

<Tag: Education>

In [5]:
edut.save()

In [6]:
edut.delete()

In [7]:
edut  # still in memory!

<Tag: Education>

## Creation and Destruction with Managers

In [32]:
type(Tag.objects)  # a model manager

django.db.models.manager.Manager

In [9]:
Tag.objects.create(name='Video Games', slug='video-games')

IntegrityError: UNIQUE constraint failed: organizer_tag.slug

In [10]:
# create multiple objects in a go!
Tag.objects.bulk_create([
    Tag(name='Django', slug='django'),
    Tag(name='Mobile', slug='mobile'),
    Tag(name='Web', slug='web'),
])

IntegrityError: UNIQUE constraint failed: organizer_tag.slug

In [33]:
Tag.objects.all()

[<Tag: Django>, <Tag: Mobile>, <Tag: Video Games>, <Tag: Web>]

In [12]:
Tag.objects.all()[0]  # acts like a list

<Tag: Django>

In [13]:
type(Tag.objects.all())  # is not a list

django.db.models.query.QuerySet

In [14]:
# managers are not accessible to model instances, only to model classes!
try:
    edut.objects
except AttributeError as e:
    print(e)

Manager isn't accessible via Tag instances


## Methods of Data Retrieval

In [34]:
Tag.objects.all()

[<Tag: Django>, <Tag: Mobile>, <Tag: Video Games>, <Tag: Web>]

In [29]:
Tag.objects.count()

4

### The `get` method

In [35]:
Tag.objects.get(slug='django')

<Tag: Django>

In [17]:
type(Tag.objects.all())

django.db.models.query.QuerySet

In [18]:
type(Tag.objects.get(slug='django'))

organizer.models.Tag

In [19]:
# case-sensitive!
try:
    Tag.objects.get(slug='Django')
except Tag.DoesNotExist as e:
    print(e)

Tag matching query does not exist.


In [20]:
# the i is for case-Insensitive
Tag.objects.get(slug__iexact='DJANGO')

<Tag: Django>

In [21]:
Tag.objects.get(slug__istartswith='DJ')

<Tag: Django>

In [22]:
Tag.objects.get(slug__contains='an')

<Tag: Django>

In [23]:
# get always returns a single object
try:
    # djangO, mObile, videO-games
    Tag.objects.get(slug__contains='o')
except Tag.MultipleObjectsReturned as e:
    print(e)

get() returned more than one Tag -- it returned 3!


### The `filter` method

In [36]:
## unlike get, can fetch multiple objects
Tag.objects.filter(slug__contains='o')

[<Tag: Django>, <Tag: Mobile>, <Tag: Video Games>]

In [38]:
type(Tag.objects.filter(slug__contains='o'))

django.db.models.query.QuerySet

### Chaining Calls

In [26]:
Tag.objects.filter(slug__contains='o').order_by('-name')

[<Tag: Video Games>, <Tag: Mobile>, <Tag: Django>]

In [27]:
# first we call order_by on the manager
Tag.objects.order_by('-name')

[<Tag: Web>, <Tag: Video Games>, <Tag: Mobile>, <Tag: Django>]

In [31]:
# now we call filter on the manager, and order the resulting queryset
Tag.objects.filter(slug__contains='e').order_by('-name')

[<Tag: Web>, <Tag: Video Games>, <Tag: Mobile>]

### `values` and `values_list`

In [32]:
Tag.objects.values_list()

[(4, 'Django', 'django'), (5, 'Mobile', 'mobile'), (2, 'Video Games', 'video-games'), (6, 'Web', 'web')]

In [33]:
type(Tag.objects.values_list())

django.db.models.query.ValuesListQuerySet

In [34]:
Tag.objects.values_list('name', 'slug')

[('Django', 'django'), ('Mobile', 'mobile'), ('Video Games', 'video-games'), ('Web', 'web')]

In [35]:
Tag.objects.values_list('name')

[('Django',), ('Mobile',), ('Video Games',), ('Web',)]

In [36]:
Tag.objects.values_list('name', flat=True)

['Django', 'Mobile', 'Video Games', 'Web']

In [37]:
type(Tag.objects.values_list('name', flat=True))

django.db.models.query.ValuesListQuerySet

## Data in Memory vs Data in the Database

In [39]:
jb = Startup.objects.create(
    name='JamBon Software',
    slug='jambon-software',
    contact='django@jambonsw.com',
    description='Web and Mobile Consulting.\n'
                'Django Tutoring.\n',
    founded_date=date(2013, 1, 18),
    website='https://jambonsw.com/',
)
jb  # this output only clear because of __str__()

IntegrityError: UNIQUE constraint failed: organizer_startup.slug

In [41]:
# jb = Startup.objects.get(slug='jambon-software')
jb.founded_date

datetime.date(2013, 1, 18)

In [42]:
jb.founded_date = date(2014,1,1)
# we're not calling save() !
jb.founded_date

datetime.date(2014, 1, 1)

In [43]:
# get version in database
jb = Startup.objects.get(slug='jambon-software')
# work above is all for nought because we didn't save()
jb.founded_date

datetime.date(2013, 1, 18)

## Connecting Data through Relations

In [45]:
djt = Post.objects.create(
    title='Django Training',
    slug='django-training',
    text=(
        "Learn Django in a classroom setting "
        "with JamBon Software."),
)
djt

<Post: Django Training:2017-10-31>

In [46]:
djt.pub_date = date(2013, 1, 18)
djt.save()
djt

<Post: Django Training:2013-01-18>

In [48]:
type(djt.tags)

django.db.models.fields.related.create_many_related_manager.<locals>.ManyRelatedManager

In [49]:
type(djt.startups)

django.db.models.fields.related.create_many_related_manager.<locals>.ManyRelatedManager

In [50]:
djt.tags.all()

[]

In [51]:
djt.startups.all()

[]

In [52]:
django = Tag.objects.get(slug__contains='django')
djt.tags.add(django)
djt.tags.all()

[<Tag: Django>]

In [46]:
django.blog_posts.all()  # a "reverse" relation

[<Post: Django Training on 2013-01-18>]

In [47]:
django.startup_set.add(jb)  # a "reverse" relation
django.startup_set.all()

[<Startup: JamBon Software>]

In [48]:
jb.tags.all()  # the "forward" relation

[<Tag: Django>]

In [49]:
# on more time, for repetition!
djt

<Post: Django Training on 2013-01-18>

In [50]:
# "forward" relation
djt.startups.add(jb)
djt.startups.all()

[<Startup: JamBon Software>]

In [51]:
jb.blog_posts.all()  # "reverse" relation

[<Post: Django Training on 2013-01-18>]